In [1]:
import pandas
import os

In [2]:
import utils

In [65]:
import os
from math import radians, cos, sin, asin, sqrt
import sys
import time

import sys
sys.path.append('/Users/gustaveronteix/Documents/Projets/Projets Code/batch7_beges/datasets/chorus-dt')

import requests
import dill
import urllib.parse
import pandas as pd
from tqdm import tqdm
from utils.config import CONFIG

from utils import dataset
from utils.resolvers import HardcodesResolver, GeocodingApiResolver

tqdm.pandas()


def compute_distance(x, n):
    lon0 = x["coords_place_0_lon"]
    lat0 = x["coords_place_0_lat"]
    lon1 = x["coords_place_1_lon"]
    lat1 = x["coords_place_1_lat"]
    lon2 = x["coords_place_2_lon"]
    lat2 = x["coords_place_2_lat"]
    
    ### CALC FIRST HALF
    
    lon0, lat0, lon1, lat1 = map(radians, [lon0, lat0, lon1, lat1])

    # haversine formula
    dlon = lon1 - lon0
    dlat = lat1 - lat0
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in miles. Use 6371 for kilometers
    
    d1 = c * r
    
    ### CALC SECOND HALF 
    
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in miles. Use 6371 for kilometers
    
    d2  = c * r
    
    return d1, d2

def compute_distance(df, n):
    
    coord_lon_str = 'coords_place_'+str(n)+'_lon'
    coord_lat_str = 'coords_place_'+str(n)+'_lat'
    coord_lon_p_str = 'coords_place_'+str(n-1)+'_lon'
    coord_lat_p_str = 'coords_place_'+str(n-1)+'_lat'
    
    lon0 = df[coord_lon_str]
    lat0 = df[coord_lat_str]
    lon1 = df[coord_lon_p_str]
    lat1 = df[coord_lat_p_str]

    
    ### CALC FIRST HALF
        
    lon0 = np.deg2rad(lon0)
    lon1 = np.deg2rad(lon1)
    lat0 = np.deg2rad(lat0)
    lat1 = np.deg2rad(lat1)

    # haversine formula
    dlon = lon1 - lon0
    dlat = lat1 - lat0
    a = np.sin((lat1 - lat0)/2) ** 2 + np.cos(lat1) * np.cos(lat0) * np.sin((lon1 - lon0)/ 2)** 2
    
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of earth in miles. Use 6371 for kilometers
    
    return c*r


if __name__ == "__main__":
    data = dataset.load_data("./data/raw/sample-avion-train.csv")
    places, trips = dataset.get_places_and_trips(data, ["AU"])

    # Splitting codes / name
    places[["code_1", "name", "temp", "code_2"]] = places["place"].str.extract(
        "([^-]*)\s-\s([^\[]*)\s?([\[](.*)[\]])?", expand=True
    )
    places.drop(columns=["temp"], inplace=True)

    places["resolved"] = False
    places["resolved_through_uic_code"] = False
    places["resolved_through_insee_code"] = False
    places["resolved_through_tvs_code"] = False
    places["resolved_through_iata_code"] = False
    places["resolved_through_gmap_api"] = False
    places["lon"] = None
    places["address"] = ""
    places["lat"] = None

    hardcode_resolver = HardcodesResolver()
    places = places.progress_apply(hardcode_resolver.resolve, axis=1)

    # Initiate geocoder
    GMAP_API_KEY = CONFIG["api_keys"]["gmap"]
    geocoding_api_resolver = GeocodingApiResolver(api_name="gmap", api_key=GMAP_API_KEY)

    # Apply the geocoder with delay using the rate limiter:
    places.loc[~places["resolved"], :] = places[~places["resolved"]].progress_apply(
        geocoding_api_resolver.resolve, axis=1
    )
    geocoding_api_resolver.save_cache()
    for c in places:
        if "resolved_through_" in c:
            print(c, places[c].sum())

    places["lat"] = places["lat"].astype(float)
    places["lon"] = places["lon"].astype(float)
    places_dict = places.set_index("place").to_dict()

    for place_index in [0, 1, 2]:
        
        trips["coords_place_%d_lat" % place_index] = trips[
                "trip_place_%d" % place_index
            ].apply(lambda x: places_dict["lat"][x])
        trips["coords_place_%d_lon" % place_index] = trips[
                "trip_place_%d" % place_index
            ].apply(lambda x: places_dict["lon"][x])
        trips["coords_place_%d" % place_index] = (
                trips["coords_place_%d_lon" % place_index].astype(str)
                + ";"
                + trips["coords_place_%d_lat" % place_index].astype(str)
            )
        trips["place_%d_count" % place_index] = trips[
                "trip_place_%d" % place_index
            ].apply(lambda x: places_dict["total"][x])

    trips["distance_1"] = compute_distance(trips, 1)
    trips["distance_2"] = compute_distance(trips, 2)
    trips["distance"] = trips["distance_1"] + trips["distance_2"]

    places.to_csv("./data/clean/places.csv", index=False)
    trips.to_csv("./data/clean/trips.csv", index=False)


Prestation types: 
['AU']
Unique places src:  128
Unique places dst:  129
Unique places stop:  1
Unique places:  137
Unique trips:  325


100%|██████████| 26/26 [00:00<00:00, 1141.30it/s]


resolved_through_uic_code 0
resolved_through_insee_code 36
resolved_through_tvs_code 0
resolved_through_iata_code 75
resolved_through_gmap_api 25


KeyError: 'No stop'

In [63]:
places

,place,src,dst,stop,total,code_1,name,code_2,resolved,resolved_through_uic_code,resolved_through_insee_code,resolved_through_tvs_code,resolved_through_iata_code,resolved_through_gmap_api,lon,address,lat
0,75056 - PARIS,295.0,301.0,0.0,596.0,75056,PARIS,NaN,True,False,False,False,False,True,2.352222,"Paris, France",48.856614
1,IA_PAR - PARIS,71.0,81.0,0.0,152.0,IA_PAR,PARIS,NaN,True,False,False,False,False,True,2.352222,"Paris, France",48.856614
2,31555 - TOULOUSE,70.0,53.0,0.0,123.0,31555,TOULOUSE,NaN,True,False,True,False,False,False,1.431673,,43.596381
3,IA_LIS - LISBONNE,59.0,56.0,0.0,115.0,IA_LIS,LISBONNE,NaN,True,False,False,False,True,False,-9.135920,,38.781300
4,DEP075 - PARIS,52.0,55.0,0.0,107.0,DEP075,PARIS,NaN,True,False,False,False,False,True,2.352222,"Paris, France",48.856614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,54395 - NANCY,0.0,4.0,0.0,4.0,54395,NANCY,NaN,True,False,True,False,False,False,6.175883,,48.690200
133,IA_LEJ - LEIPZIG,0.0,4.0,0.0,4.0,IA_LEJ,LEIPZIG,NaN,True,False,False,False,True,False,12.236389,,51.423889
134,IA_MDE - MEDELLIN,0.0,3.0,0.0,3.0,IA_MDE,MEDELLIN,NaN,True,False,False,False,True,False,-75.423100,,6.164540
135,IA_TNG - TANGER,0.0,3.0,0.0,3.0,IA_TNG,TANGER,NaN,True,False,False,False,True,False,-5.916890,,35.726898


In [ ]:
trips

In [6]:
import numpy as np

data = dataset.load_data("./data/raw/sample-avion-train.csv")
places, trips = dataset.get_places_and_trips(data, ["A"])

Prestation types: 
['A']
Unique places src:  230
Unique places dst:  228
Unique places stop:  1
Unique places:  241
Unique trips:  753


In [11]:
if __name__ == "__main__":
    data = dataset.load_data("./data/raw/sample-avion-train.csv")
    places, trips = dataset.get_places_and_trips(data, ["A"])

    # Splitting codes / name
    places[["code_1", "name", "temp", "code_2"]] = places["place"].str.extract(
        "([^-]*)\s-\s([^\[]*)\s?([\[](.*)[\]])?", expand=True
    )
    places.drop(columns=["temp"], inplace=True)

    places["resolved"] = False
    places["resolved_through_uic_code"] = False
    places["resolved_through_insee_code"] = False
    places["resolved_through_tvs_code"] = False
    places["resolved_through_iata_code"] = False
    places["resolved_through_gmap_api"] = False
    places["lon"] = None
    places["address"] = ""
    places["lat"] = None

    hardcode_resolver = HardcodesResolver()
    places = places.progress_apply(hardcode_resolver.resolve, axis=1)

Prestation types: 
['A']
Unique places src:  230
Unique places dst:  228
Unique places stop:  1
Unique places:  241
Unique trips:  753


100%|██████████| 241/241 [00:00<00:00, 1906.63it/s]


In [17]:
def compute_distance(df, n):
    
    coord_lon_str = 'coords_place_'+str(n)+'_lon'
    coord_lat_str = 'coords_place_'+str(n)+'_lat'
    coord_lon_m_str = 'coords_place_'+str(n-1)+'_lon'
    coord_lat_m_str = 'coords_place_'+str(n-1)+'_lat'
    
    lon0 = df[coord_lon_str]
    lat0 = df[coord_lat_str]
    lon1 = df[coord_lon_m_str]
    lat1 = df[coord_lat_m_str]

    
    ### CALC FIRST HALF
    
    lon0, lat0, lon1, lat1 = map(radians, [lon0, lat0, lon1, lat1])

    # haversine formula
    dlon = lon1 - lon0
    dlat = lat1 - lat0
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in miles. Use 6371 for kilometers
    
    return c*r

In [13]:
places[~places['resolved_through_iata_code']]

,place,src,dst,stop,total,code_1,name,code_2,resolved,resolved_through_uic_code,resolved_through_insee_code,resolved_through_tvs_code,resolved_through_iata_code,resolved_through_gmap_api,lon,address,lat
3,- PARIS ORLY [ORY],364.0,374.0,0.0,738.0,,PARIS ORLY,ORY,False,False,False,False,False,False,None,,None
4,- Arpt d'Orly,314.0,313.0,0.0,627.0,,Arpt d'Orly,NaN,False,False,False,False,False,False,None,,None
10,75056 - PARIS,114.0,109.0,0.0,223.0,75056,PARIS,NaN,False,False,False,False,False,False,None,,None
28,IA_PAR - PARIS,43.0,45.0,0.0,88.0,IA_PAR,PARIS,NaN,False,False,False,False,False,False,None,,None
31,31555 - TOULOUSE,41.0,44.0,0.0,85.0,31555,TOULOUSE,NaN,True,False,True,False,False,False,1.43167293364,,43.5963814303
41,- Paris Orly Airport,27.0,21.0,0.0,48.0,,Paris Orly Airport,NaN,False,False,False,False,False,False,None,,None
77,97120 - POINTE-À-PITRE,11.0,4.0,0.0,15.0,97120,POINTE-À-PITRE,NaN,True,False,True,False,False,False,-61.5357180881,,16.2386006963
79,92062 - PUTEAUX,11.0,7.0,0.0,18.0,92062,PUTEAUX,NaN,True,False,True,False,False,False,2.23834178314,,48.8837094969
82,- Blaise Diagne International Airport,10.0,10.0,0.0,20.0,,Blaise Diagne International Airport,NaN,False,False,False,False,False,False,None,,None
98,IA_MLH - MULHOUSE,8.0,8.0,0.0,16.0,IA_MLH,MULHOUSE,NaN,False,False,False,False,False,False,None,,None


In [29]:
a = places.loc[0]
with open(("./data/prepared/tvs_codes.pkl"), "rb") as file_id:
    tvs_codes = dill.load(file_id)

In [31]:
a["code_1"]

'TR_FRPNO'

In [32]:
a["code_1"] in tvs_codes['Longitude WGS84']

True

In [33]:
a["lon"] = tvs_codes['Longitude WGS84'][a["code_1"]]
a["lat"] = tvs_codes['Latitude WGS84'][a["code_1"]]

/Users/gustaveronteix/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/gustaveronteix/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
a

place                          TR_FRPNO - Paris Nord
src                                             3212
dst                                             3190
stop                                             194
total                                           6596
code_1                                      TR_FRPNO
name                                      Paris Nord
code_2                                           NaN
resolved                                       False
resolved_through_uic_code                      False
resolved_through_insee_code                    False
resolved_through_tvs_code                      False
resolved_through_iata_code                     False
resolved_through_gmap_api                      False
lon                                          2.35515
address                                             
lat                                          48.8802
Name: 0, dtype: object

In [17]:
hardcode_resolver = HardcodesResolver()
tvs_codes = hardcode_resolver.tvs_codes

In [21]:
tvs_codes['Longitude WGS84']['TR_FRPNO']

2.3551509999999998